<a href="https://colab.research.google.com/github/AkbarBunyad/ML_Foundations_with_tf/blob/main/Notes/using_NLP_to_build_sarcasm_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O /tmp/sarcasm.json

--2024-02-25 16:10:48--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.207, 172.253.117.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.1s    

2024-02-25 16:10:49 (46.6 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [ ]:
import json

with open("/tmp/sarcasm.json", 'r') as file:
  datastore = json.load(file)

sentences = []
labels = []
urls = []

for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(oov_token = '<OOV>')
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
print(len(word_index))
print(word_index)
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding = 'post')
print(padded[0])
print(padded.shape)
#print(len(sentences))

29657
{'<OOV>': 1, 'to': 2, 'of': 3, 'the': 4, 'in': 5, 'for': 6, 'a': 7, 'on': 8, 'and': 9, 'with': 10, 'is': 11, 'new': 12, 'trump': 13, 'man': 14, 'from': 15, 'at': 16, 'about': 17, 'you': 18, 'this': 19, 'by': 20, 'after': 21, 'up': 22, 'out': 23, 'be': 24, 'how': 25, 'as': 26, 'it': 27, 'that': 28, 'not': 29, 'are': 30, 'your': 31, 'his': 32, 'what': 33, 'he': 34, 'all': 35, 'just': 36, 'who': 37, 'has': 38, 'will': 39, 'more': 40, 'one': 41, 'into': 42, 'report': 43, 'year': 44, 'why': 45, 'have': 46, 'area': 47, 'over': 48, 'donald': 49, 'u': 50, 'day': 51, 'says': 52, 's': 53, 'can': 54, 'first': 55, 'woman': 56, 'time': 57, 'like': 58, 'her': 59, "trump's": 60, 'old': 61, 'no': 62, 'get': 63, 'off': 64, 'an': 65, 'life': 66, 'people': 67, 'obama': 68, 'now': 69, 'house': 70, 'still': 71, "'": 72, 'women': 73, 'make': 74, 'was': 75, 'than': 76, 'white': 77, 'back': 78, 'my': 79, 'i': 80, 'clinton': 81, 'down': 82, 'if': 83, '5': 84, 'when': 85, 'world': 86, 'could': 87, 'we': 8

In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 16
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_size = 20000

In [ ]:
training_sentences = sentences[0: training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0: training_size]
testing_labels = labels[training_size:]

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen = max_length,
                                padding = padding_type, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length,
                                padding = padding_type, truncating = trunc_type)

In [ ]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 16, 16)            160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160433 (626.69 KB)
Trainable params: 160433 (626.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
625/625 - 3s - loss: 0.4917 - accuracy: 0.7617 - val_loss: 0.3602 - val_accuracy: 0.8450 - 3s/epoch - 5ms/step
Epoch 2/30
625/625 - 2s - loss: 0.2689 - accuracy: 0.8910 - val_loss: 0.3473 - val_accuracy: 0.8462 - 2s/epoch - 4ms/step
Epoch 3/30
625/625 - 4s - loss: 0.1975 - accuracy: 0.9212 - val_loss: 0.3674 - val_accuracy: 0.8492 - 4s/epoch - 7ms/step
Epoch 4/30
625/625 - 2s - loss: 0.1547 - accuracy: 0.9420 - val_loss: 0.4065 - val_accuracy: 0.8486 - 2s/epoch - 4ms/step
Epoch 5/30
625/625 - 2s - loss: 0.1261 - accuracy: 0.9542 - val_loss: 0.4560 - val_accuracy: 0.8402 - 2s/epoch - 3ms/step
Epoch 6/30
625/625 - 2s - loss: 0.1041 - accuracy: 0.9635 - val_loss: 0.5129 - val_accuracy: 0.8360 - 2s/epoch - 3ms/step
Epoch 7/30
625/625 - 2s - loss: 0.0856 - accuracy: 0.9718 - val_loss: 0.5689 - val_accuracy: 0.8335 - 2s/epoch - 3ms/step
Epoch 8/30
625/625 - 2s - loss: 0.0721 - accuracy: 0.9758 - val_loss: 0.6454 - val_accuracy: 0.8262 - 2s/epoch - 4ms/step
Epoch 9/30
625/625 - 2s 

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(training_padded[0])
print(training_sentences[2])
print(labels[2])
#print(reverse_word_index)

[ 328    1  799 3405 2404   47  389 2214    1    6 2614 8863    0    0
    0    0]
mom starting to fear son's web series closest thing she will have to grandchild
1
{1: '<OOV>', 2: 'to', 3: 'of', 4: 'the', 5: 'in', 6: 'for', 7: 'a', 8: 'on', 9: 'and', 10: 'with', 11: 'is', 12: 'new', 13: 'trump', 14: 'man', 15: 'from', 16: 'at', 17: 'about', 18: 'you', 19: 'by', 20: 'this', 21: 'after', 22: 'be', 23: 'up', 24: 'out', 25: 'that', 26: 'how', 27: 'as', 28: 'it', 29: 'not', 30: 'are', 31: 'your', 32: 'what', 33: 'his', 34: 'all', 35: 'he', 36: 'will', 37: 'who', 38: 'just', 39: 'has', 40: 'more', 41: 'one', 42: 'year', 43: 'into', 44: 'report', 45: 'have', 46: 'why', 47: 'over', 48: 'area', 49: 'u', 50: 'donald', 51: 'says', 52: 'day', 53: 'can', 54: 's', 55: 'first', 56: 'woman', 57: 'time', 58: 'like', 59: 'get', 60: 'old', 61: 'no', 62: 'her', 63: "trump's", 64: 'off', 65: 'now', 66: 'an', 67: 'obama', 68: 'life', 69: 'people', 70: 'women', 71: 'house', 72: "'", 73: 'was', 74: 'still', 

In [ ]:
sentence = ["granny starting to fear spiders in the garden might be real", "game of thrones season finale showing this sunday night"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

1/1 [==============================] - 0s 86ms/step
[[9.9126160e-01]
 [3.1779046e-10]]


In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [ ]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>